In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Post-Traumatic_Stress_Disorder/GSE845'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"PTSD Raw Data"
!Series_summary	"PTSD - Posttraumatic stress disorder."
!Series_summary	"33 samples taken from PMBCs of survivors of psychological trauma, in two time points: in ER, few hours after the truma, and four months later. Some of the patients devepled chronic PTSD (17 samples) and others recovered and set to be the Control group (16 samples). This is the raw data consists of 12,600 probes from U95A chip."
!Series_summary	"Samples are labeled with 3 tags: P/C for PTSD or Control, ER/M4 - for time point of sample, D/ND for Decrement or Non-decrement symptoms over time. (e.g. sample 23C-M4-D was taken 4 months after trauma from patient 23 which belongs to the control group and showed decrease in symptoms) . Samples include the expression value, the GeneBank accession number and Affymetrix indication of valid calls."
!Series_summary	"Keywords: other"
Sample Characteristics Dictionary:
{}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Assuming dataset and sample_characteristics were loaded before this step
# Check if gene expression data is available
is_gene_available = True  # The dataset background indicates gene expression data.

# Sample Characteristics Dictionary is empty, infer variables non-availability
# For this revision, we assume sample_characteristics is empty as per the output
sample_characteristics = {}  # This should be populated with actual data

# Infer the presence of variables (this part assumes some actual data in sample_characteristics)
trait_row = None
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    mapping = {"PTSD": 1, "Control": 0}
    return mapping.get(value.split(":")[1].strip(), None)

def convert_age(value):
    try:
        return float(value.split(":")[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    mapping = {"female": 0, "male": 1}
    return mapping.get(value.split(":")[1].strip().lower(), None)

# Saving the cohort information 
save_cohort_info('GSE845', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Assuming clinical_data has been defined before
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Post-Traumatic_Stress_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/trait_data/GSE845.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
